# RAG System in images data


# using google gemini embedding model for retriever

# Face Detection With Embedding

In [6]:
!pip install -qU facenet-pytorch

In [ ]:
!pip install -qU pillow

# libraries of image processing and create embaddings

In [8]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1
model = InceptionResnetV1(pretrained='vggface2').eval()
model

# function to preprocess the image
# function to create the embeddings

In [10]:
# preprocessing function to transform the image into tensor
def preprocess_image(image_path):
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize((224,224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    return preprocess(image).unsqueeze(0)
# function to create image embeddings
def create_image_embedding(image_path):
    try:
      input_tensor = preprocess_image(image_path)
      with torch.no_grad():
        embedding = model(input_tensor) # embedding important line
      return embedding.squeeze().tolist()
    except Exception as e:
      print("Error",e)
      return None

# To create image directory

In [32]:
!mkdir images

# creating the embeddings of one image

In [11]:
image_path = "/content/images/Ahmad-1.jpg"
A1 = create_image_embedding(image_path)

# to see the embeddings and dimension

In [ ]:
print(len(A1))
print("image embedding",A1)

# Making the embeddings of the images

In [13]:
from re import M
A1=create_image_embedding("/content/images/Ahmad-1.jpg")
A2=create_image_embedding("/content/images/Ahmad-2.jpg")
A3=create_image_embedding("/content/images/Ahmad-3.jpg")
M1=create_image_embedding("/content/images/Muneer-1.png")
M2=create_image_embedding("/content/images/Muneer-2.png")
M3=create_image_embedding("/content/images/Muneer-3.png")

# installing Milvus database

In [ ]:
!pip install -qU milvus-lite
!pip install pymilvus

# creating Milvus database

In [15]:
from pymilvus import MilvusClient
client = MilvusClient("./milvus_demo.db")

# creating the Milvus vector database

In [16]:
from pymilvus import MilvusClient
import numpy as np

client = MilvusClient("./milvus_demo.db")
client.create_collection(
    collection_name="images",
    dimension = 512 # the vector we will use in this demo has 384 dimensions
)

# Making a database of our data means metadata

In [17]:
data = [
    {"id": 1, "person_name": "Ahmad","vector": A1 },
    {"id": 2, "person_name": "Ahmad","vector": A2 },
    {"id": 3, "person_name": "Ahmad","vector": A3 },
    {"id": 4, "person_name": "Muneer","vector": M1 },
    {"id": 5, "person_name": "Muneer","vector": M2 },
    {"id": 6, "person_name": "Muneer","vector": M3 }

]

# inserting embeddings int the Milvus vector store

In [18]:
res = client.insert(
    collection_name="images",
    data=data,
    partition_name="images"
)

# testing image from embeddings

In [19]:
ress = client.search(
    collection_name="images",
    data = [A1],
    limit = 1,
    output_fields=["person_name"],
)
print(ress)

data: ["[{'id': 1, 'distance': 1.0, 'entity': {'person_name': 'Ahmad'}}]"]


# creating embeddings of the new image

In [20]:
A4=create_image_embedding("/content/images/Ahmad.jpg")

In [ ]:
A4

# Testing with the new image

In [22]:
resul = client.search(
    collection_name="images",
    data = [A4],
    limit = 3,
    output_fields=["id","person_name"],
)
print(resul)

data: ["[{'id': 5, 'distance': 0.5289016962051392, 'entity': {'person_name': 'Muneer', 'id': 5}}, {'id': 6, 'distance': 0.3440612256526947, 'entity': {'person_name': 'Muneer', 'id': 6}}, {'id': 4, 'distance': 0.15956392884254456, 'entity': {'person_name': 'Muneer', 'id': 4}}]"]
